# Homework #3

## Convolutionary Neural Network Implementation for HOMEWORK 3

In [10]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras.optimizers import SGD
from keras.utils import np_utils

# input params
batch_size = 32
nb_classes = 10
nb_epoch = 1000
data_augmentation = False 

# input image dimensions
img_rows, img_cols = 50, 37

# the dataset values are single dimension/channel of intensity
img_channels = 1

# load dataset
givenX_train = np.load('X_train.npy')
givenY_train = np.load('y_train.npy')
givenX_test = np.load('X_test.npy')

# split training set into subsets
trainX, testX, trainY, testY = train_test_split(givenX_train, givenY_train, test_size=0.25, random_state=42)

# use PCA to reduce the train/test data to eigenfaces
n_components = 150
pca = PCA(n_components = n_components, svd_solver='randomized', whiten=True).fit(trainX)
trainXPCA = pca.transform(trainX)
testXPCA = pca.transform(testX)

# the data, shuffled and split between train and test sets
print('X_train shape:', trainXPCA.shape)
print('X_test shape:', testXPCA.shape)
print(trainXPCA.shape[0], 'train samples')
print(testXPCA.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(trainY, nb_classes)
Y_test = np_utils.to_categorical(testY, nb_classes)
print('Y_train shape:', Y_train.shape)
print('Y_test shape:', Y_test.shape)

# normalize
#trainXPCA = trainXPCA.astype('float32')
#testXPCA = testXPCA.astype('float32')
#trainXPCA /= 255
#testXPCA /= 255

# reshape to fit data model (considering the sample and channel dimensions as well)
reshapedTrainX = trainXPCA.reshape((trainXPCA.shape[0],) + (1, trainXPCA.shape[1]))
reshapedTestX = testXPCA.reshape((testXPCA.shape[0],) + (1, testXPCA.shape[1]))
#reshapedTrainY = Y_train.reshape((-1,1))
#reshapedTestY = Y_test.reshape((-1,1))

# sequentially apply filters
model = Sequential()

# declare convolution shape with 32 convolution filters
model.add(Convolution1D(32, 1, border_mode='same', input_shape=(1, trainXPCA.shape[1])))
model.add(Activation('relu'))
model.add(Convolution1D(32, 1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_length=1))
model.add(Dropout(0.25))

model.add(Convolution1D(64, 1, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution1D(64, 1))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_length=1))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# train the model using SGD + momentum
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(reshapedTrainX, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(reshapedTestX, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(reshapedTrainX)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(reshapedTrainX, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=reshapedTrainX.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(reshapedTestX, Y_test))


('X_train shape:', (724, 150))
('X_test shape:', (242, 150))
(724, 'train samples')
(242, 'test samples')
('Y_train shape:', (724, 10))
('Y_test shape:', (242, 10))


TypeError: an integer is required